In [26]:
import numpy as np
import torch
import torch.nn.functional as F
print(torch.cuda.is_available())
import matplotlib.pyplot as plt
%matplotlib inline

False


In [2]:
words = open('./names.txt', 'r').read().splitlines()
words[:9]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper']

In [3]:
fucking_huge_word = ''.join(words)
chars = sorted(list(set(fucking_huge_word)))

In [4]:
stoi = {s : i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i : s for s, i in stoi.items()}
print(stoi)
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
block_size = 3
X, Y = [], []
# emma 
# ... => e
# ..e => m
# .em => m
# emm => a
# mma => .
for w in words[:5]:
    w = w + '.'
    context = [0] * block_size
    for ch in w:
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X, Y = torch.tensor(X), torch.tensor(Y)

In [6]:
# Build embedding matrix and then we gonna tune in the embedding matrix
C = torch.rand((27, 2))
C

tensor([[0.0578, 0.5870],
        [0.1527, 0.0844],
        [0.0780, 0.9524],
        [0.8757, 0.7942],
        [0.2902, 0.3848],
        [0.6053, 0.4643],
        [0.7809, 0.2603],
        [0.6612, 0.7750],
        [0.8563, 0.1648],
        [0.2362, 0.4324],
        [0.8589, 0.1081],
        [0.9131, 0.7159],
        [0.8790, 0.8840],
        [0.8719, 0.9432],
        [0.2932, 0.0300],
        [0.7886, 0.7492],
        [0.0945, 0.0063],
        [0.8503, 0.6280],
        [0.9222, 0.3967],
        [0.9339, 0.1590],
        [0.7604, 0.3986],
        [0.1265, 0.0588],
        [0.3818, 0.6440],
        [0.2085, 0.3112],
        [0.8285, 0.0959],
        [0.1233, 0.4152],
        [0.7489, 0.2053]])

In [7]:
C.shape

torch.Size([27, 2])

In [8]:
C[[0, 1, 2]]

tensor([[0.0578, 0.5870],
        [0.1527, 0.0844],
        [0.0780, 0.9524]])

In [9]:
# we can think this as both the first layer of a huge neural network or a DICTIONARY
C[stoi[itos[5]]] == F.one_hot(torch.tensor(5), num_classes = 27).float() @ C

tensor([True, True])

In [10]:
X.shape

torch.Size([32, 3])

In [11]:
# row 5,6,7
C[[5,6,7]]
C[torch.tensor([5,5,5,5,6])]

tensor([[0.6053, 0.4643],
        [0.6053, 0.4643],
        [0.6053, 0.4643],
        [0.6053, 0.4643],
        [0.7809, 0.2603]])

In [12]:
display(C[X].shape)
display(X.shape)

torch.Size([32, 3, 2])

torch.Size([32, 3])

In [13]:
print(X[3][1])
print(C[X][3][1])
print(C[X[3][1]])

tensor(13)
tensor([0.8719, 0.9432])
tensor([0.8719, 0.9432])


In [14]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [15]:
emb[3] == C[X[3]]

tensor([[True, True],
        [True, True],
        [True, True]])

In [16]:
X[3]

tensor([ 5, 13, 13])

In [17]:
print(C[5])
print(C[13])

tensor([0.6053, 0.4643])
tensor([0.8719, 0.9432])


In [18]:
W1 = torch.randn((2 * 3, 100)) # 2d embedding and 3 of them
b1 = torch.randn(100) # for 100 of them

In [19]:
emb.shape

torch.Size([32, 3, 2])

In [20]:
emb = emb.view(32, -1)
emb.shape

torch.Size([32, 6])

In [21]:
h = emb @ W1 + b1

In [22]:
h

tensor([[-0.7055, -0.1611,  0.8827,  ...,  0.3002,  2.3732, -1.7284],
        [-1.2261,  0.1362, -0.0391,  ...,  0.2978,  2.0010, -1.1373],
        [-0.8631,  0.3655, -0.1330,  ..., -0.5366,  2.2013, -1.0891],
        ...,
        [-1.6110, -0.3572,  0.5121,  ...,  0.8182,  2.1828, -0.0986],
        [-0.1522,  0.5805,  0.6638,  ...,  0.6593,  0.6412, -1.5293],
        [-1.8416, -0.3464,  0.9079,  ...,  2.1036,  2.1808, -1.2714]])

In [23]:
h = torch.tanh(h)

In [24]:
h

tensor([[-0.6079, -0.1597,  0.7078,  ...,  0.2915,  0.9828, -0.9389],
        [-0.8414,  0.1354, -0.0391,  ...,  0.2893,  0.9641, -0.8135],
        [-0.6978,  0.3501, -0.1322,  ..., -0.4904,  0.9758, -0.7966],
        ...,
        [-0.9233, -0.3427,  0.4716,  ...,  0.6741,  0.9749, -0.0983],
        [-0.1510,  0.5230,  0.5809,  ...,  0.5779,  0.5657, -0.9103],
        [-0.9510, -0.3332,  0.7201,  ...,  0.9707,  0.9748, -0.8542]])

In [25]:
h.shape

torch.Size([32, 100])